### 0. Load environment variables
#### Case 1) If you use OpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'model_name' = XXXXXXXXXXXXXXX

##### Case 2) If you use AzureOpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_TYPE' = XXXXXXXXXXXXXXX
- 'OPENAI_API_VERSION' = XXXXXXXXXXXXXXX
- 'OPENAI_API_BASE' = XXXXXXXXXXXXXXX
- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'deployment_name' = XXXXXXXXXXXXXXX

In [1]:
import os
import sys  
from dotenv import load_dotenv
sys.path.extend(["..", '../..'])  
_ = load_dotenv('.env')


model_name = os.getenv('deployment_name') if os.getenv('OPENAI_API_TYPE') == 'azure' else os.getenv('model_name')
if os.getenv('OPENAI_API_TYPE') == 'azure':
    deployment_name = model_name

### 1. Create a scratchpad-compressing chain (using LCEL)

In [2]:
from langchain import hub
agent_scratchpad_compression_prompt = hub.pull("jet-taekyo-lee/agent-scratchpad-compression")

from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
llm = AzureChatOpenAI(deployment_name=deployment_name, temperature=0) if os.getenv('OPENAI_API_TYPE') == 'azure' else ChatOpenAI(model_name=model_name, temperature=0)

agent_scratchpad_compression_chain = agent_scratchpad_compression_prompt | llm 

### 2. Fetch the prompt and create a chain (using LCEL)

In [9]:
from langchain import hub
prompt = hub.pull("jet-taekyo-lee/rag-style-react")

from utils.agent_components.configurations import Configurations
from utils.agent_tools.tools.web_search_tools import get_web_search_tools
from utils.agent_tools.tools.weather_tools import GetFromOpenWeatherMap
from utils.agent_tools.tools.time_tools import GetFromDatetimeModule
from utils.agent_tools.tools.python_repl_tools import GetLangChainPythonRepl
from langchain.tools.render import render_text_description_and_args
config = Configurations(provider='AzureChatOpenAI')
search_tool = get_web_search_tools(config)
weather_tool = GetFromOpenWeatherMap()
time_tool = GetFromDatetimeModule()
python_repl_tool = GetLangChainPythonRepl()
tools = [search_tool, weather_tool, time_tool, python_repl_tool]

from langchain.prompts import ChatPromptTemplate
prompt = prompt.partial(
        tools=render_text_description_and_args(tools),
        tool_names=", ".join([t.name for t in tools]),
)


from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
llm = AzureChatOpenAI(deployment_name=deployment_name, temperature=0) if os.getenv('OPENAI_API_TYPE') == 'azure' else ChatOpenAI(model_name=model_name, temperature=0)

from langchain.schema.runnable import RunnablePassthrough
from utils.custom_output_parsers.agent_intermediatte_steps import format_intersteps_to_compressed_scratchpad
from langchain.agents.output_parsers import JSONAgentOutputParser

rag_style_react_agent = ( RunnablePassthrough.assign( compressed_agent_scratchpad = lambda x : format_intersteps_to_compressed_scratchpad(x["intermediate_steps"], agent_scratchpad_compression_chain) )
    | prompt | llm | JSONAgentOutputParser()
)

from langchain.agents import AgentExecutor
rag_style_react_agent_executor = AgentExecutor(agent=rag_style_react_agent, tools=tools, verbose=True)


### Get a response

In [10]:
rag_style_react_agent_executor.invoke({'input':'What is the weather in new york?'})



> Entering new AgentExecutor chain...
Thought: The user wants to know the current weather in New York. I don't need to know the current time for this task, I can directly use the GetFromOpenWeatherMap tool to get the current weather in New York.
Action:
```
{
"action": "GetFromOpenWeatherMap",
"action_input": {"IANA_timezone": "America/New_York", "Days_from_now": 0}
}
```timezone: New York County, current time(year-month-day): 2023-11-12-17-48, temperature: 5.340000000000032, humidity: 45, wind_speed: 4.36, weather_summary: Expect a day of partly cloudy with clear spells.Thought: The user is asking for the current weather in New York. However, the additional information provided already gives the current weather in New York on November 12th, 2023. Therefore, I don't need to use any tools to find the weather. I can directly provide the answer.

Action:
```
{
"action": "Final Answer",
"action_input": "The current weather in New York on November 12th, 2023, is partly cloudy with clear s

{'input': 'What is the weather in new york?',
 'output': 'The current weather in New York on November 12th, 2023, is partly cloudy with clear spells, a temperature of 5.34°C, humidity at 45%, and a wind speed of 4.36 m/s.'}